In [1599]:
# import mysql.connector as connection
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [1600]:
# host = 'localhost'
# username = 'root'
# passwd = ''
# database = 'db_ml_dummies'

In [1601]:
# try:
#     db = connection.connect(host=host, database=database,user=username, passwd=passwd,use_pure=True)
#     query = "SELECT * FROM tasks;"
#     df = pd.read_sql(query, db)
#     db.close()
# except Exception as e:
#     db.close()
#     print(str(e))

In [1602]:
df = pd.read_csv('inserted_tasks.csv')

In [1603]:
df.head()

,task_type,number_of_peoples,estimated_time_in_minutes
0,1,1,1440
1,1,2,1080
2,1,1,2160
3,1,2,1200
4,1,3,720


In [1604]:
df.describe()

,task_type,number_of_peoples,estimated_time_in_minutes
count,19.000000,19.000000,19.000000
mean,1.947368,1.842105,974.210526
std,0.848115,0.834210,750.491067
min,1.000000,1.000000,90.000000
25%,1.000000,1.000000,210.000000
50%,2.000000,2.000000,1080.000000
75%,3.000000,2.500000,1380.000000
max,3.000000,3.000000,2880.000000


In [1605]:
df.shape

(19, 3)

In [1606]:
# df.to_csv('tasks.csv', index=False)

In [1607]:
scaler = MinMaxScaler()
scaler.fit(df)
df[list(df.columns)] = scaler.transform(df)

print(df)

    task_type  number_of_peoples  estimated_time_in_minutes
0         0.0                0.0                   0.483871
1         0.0                0.5                   0.354839
2         0.0                0.0                   0.741935
3         0.0                0.5                   0.397849
4         0.0                1.0                   0.225806
5         0.0                1.0                   0.268817
6         0.0                0.5                   0.376344
7         0.5                0.0                   0.010753
8         0.5                0.0                   1.000000
9         0.5                1.0                   0.397849
10        0.5                0.5                   0.483871
11        0.5                0.0                   0.483871
12        0.5                1.0                   0.440860
13        1.0                0.0                   0.010753
14        1.0                0.0                   0.053763
15        1.0                0.0        

In [1608]:
X = df.drop(columns="estimated_time_in_minutes")
y = pd.Series(df['estimated_time_in_minutes'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1609]:
linReg = LinearRegression()

linReg.fit(X_train, y_train)

LinearRegression()

In [1610]:
# R2 Score
print(linReg.score(X_test, y_test))
# Coefficient
print(linReg.coef_)
# Intercept
print(linReg.intercept_)

-1.6708745530617186
[-0.43435859 -0.15703926]
0.6095273818454615


In [1611]:
y_pred =  linReg.predict(X_test)
print(y_pred)

[0.60952738 0.45248812 0.39234809 0.53100775]


In [1612]:
# MAE
print(mean_absolute_error(y_test, y_pred))
# MSE
print(mean_squared_error(y_test, y_pred))
# RMSE
print(math.sqrt(mean_squared_error(y_test, y_pred)))

0.14425481370342597
0.022234127392813482
0.1491111243094005


In [1613]:
request = {
    'task_type': [3],
    'number_of_peoples': [3],
    'estimated_time_in_minutes': [0]
}

# Create Pandas DataFrame
request = pd.DataFrame.from_dict(request)
# Scale the request
request[list(request.columns)] = scaler.transform(request)
# Remove unused column
request = request.drop(columns='estimated_time_in_minutes')

In [1614]:
result = linReg.predict(request)
print(result)

[0.01812953]


In [1615]:
originalRequest = request
originalRequest['estimated_time_in_minutes'] = result[0]
scaledRequest = scaler.inverse_transform(originalRequest);

print(scaledRequest[0, 2])

140.5813953488369


In [1616]:
# save the model to disk
modelname = 'finalized_model.sav'
pickle.dump(linReg, open(modelname, 'wb'))

scalername = 'scaler.sav'
pickle.dump(scaler, open(scalername, 'wb'))